In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from easydict import EasyDict
import numpy as np
from tqdm.notebook import tqdm

#tqdm().pandas()

In [3]:
def get_datasets():
    datasets = EasyDict(dict(
        train=torchvision.datasets.MNIST(root='data/', train=True, download=True, transform=transforms.ToTensor()),
        test=torchvision.datasets.MNIST(root='data/', train=False, download=True, transform=transforms.ToTensor()),
    ))
    return datasets

In [4]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [10]:
def train_model(train_config, model, optimizer, criterion, datasets):
    loaders = EasyDict(dict(
        train=torch.utils.data.DataLoader(datasets.train, batch_size=train_config.batch_size, shuffle=True),
        test=torch.utils.data.DataLoader(datasets.test, batch_size=train_config.batch_size, shuffle=False),
    ))
    
    def train_one_epoch(epoch):
        model.train()
        pbar = tqdm(enumerate(loaders.train), total=len(loaders.train))
        for i, (images, labels) in pbar:
            images = images.to(train_config.device)
            labels = labels.to(train_config.device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.set_description(f'train epoch {epoch} loss {loss.item():.4f}')

    def test_one_epoch(epoch):
        model.eval()
        with torch.no_grad():
            total, correct = 0, 0
            pbar = tqdm(enumerate(loaders.test), total=len(loaders.test))
            for i, (images, labels) in pbar:
                images = images.to(train_config.device)
                labels = labels.to(train_config.device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                pbar.set_description(f'test  epoch {epoch} accuracy {100*correct/total:.2f}%')

    for epoch in range(train_config.num_epochs):
        train_one_epoch(epoch)
        test_one_epoch(epoch)

In [11]:
def main():
    train_config = EasyDict(dict(
        # device
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
        # training configuration
        num_epochs = 5,
        num_classes = 10,
        batch_size = 100,
        learning_rate = 1e-3,
    ))

    datasets = get_datasets()
    model = ConvNet(num_classes=train_config.num_classes).to(train_config.device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=train_config.learning_rate)
    train_model(train_config, model, optimizer, criterion, datasets)
    torch.save({'model': model.state_dict()}, 'runs/convnet_exp_1.ckpt')
main()